## Text Summarization using fine tuned facebook/bart-base model
---



***Install necessary Libraries***

In [1]:
!pip install datasets evaluate transformers rouge-score nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.2 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=ce6b02a68d30e282c711259d10a182e3215fdfa659b15f25d8e70d74c64ed640
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [2]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 7.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [3]:
!pip install nltk

***Login to Huggingfacehub***

In [4]:
from huggingface_hub import notebook_login

notebook_login()

***Installing Git-lfs***

In [5]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


***Checking Transformer Version***

In [6]:
import transformers

print(transformers.__version__)

4.38.2


In [7]:
import pandas as pd
import numpy as np

In [8]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

### Read and PreProcess the dataset for Finetuning the base model

In [10]:
input_df = pd.read_csv('curated_data_subset.csv')
input_df.head()

,Unnamed: 0,title,summary,url,date,article_content
0,0,Tencent gains approval to sell mutual funds to...,Tencent has been granted a licence from the Ch...,http://www.scmp.com/business/companies/article...,49:58.9,Traditional finance houses now being seriously...
1,1,"India testing blockchains in education, health...",India is testing blockchain applications in ed...,https://www.vccircle.com/niti-aayog-explores-b...,33:08.3,"The Indian government's policy think tank, Nit..."
2,2,Higher living wage risks robot takeover of low...,The UK Institute for Fiscal Studies has warned...,https://news.sky.com/story/ifs-living-wage-inc...,32:10.2,Increases in minimum wage levels risk raising ...
3,3,Regus WeWork may seek stock-market flotation t...,Co-working start-up WeWork may go public this ...,https://www.fool.com/investing/2018/01/03/will...,31:59.6,"Depending on whom you ask, WeWork is either a ..."
4,4,AMD poised to gain market share as Intel pound...,Intel has seen more than $11bn wiped off its m...,https://www.cnbc.com/2018/01/05/amd-is-big-win...,17:13.3,AMD is big winner from chip flaw fiasco as mor...


In [11]:
input_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       100 non-null    int64 
 1   title            100 non-null    object
 2   summary          100 non-null    object
 3   url              100 non-null    object
 4   date             100 non-null    object
 5   article_content  99 non-null     object
dtypes: int64(1), object(5)
memory usage: 4.8+ KB


In [12]:
input_df.isnull().sum()

Unnamed: 0         0
title              0
summary            0
url                0
date               0
article_content    1
dtype: int64

In [13]:
input_df.dropna(inplace=True, ignore_index= True)

In [14]:
input_df.isnull().sum()

Unnamed: 0         0
title              0
summary            0
url                0
date               0
article_content    0
dtype: int64

In [15]:
input_df.loc[29, :]

Unnamed: 0                                                        30
title              Fluenta's new gauges measure gas at extreme te...
summary            Fluenta has launched two ultrasonic flow trans...
url                https://www.oilfieldtechnology.com/product-new...
date                                                         20:05.4
article_content    Fluenta has announced the launch of its new ra...
Name: 29, dtype: object

In [16]:
input_df.rename(columns = {'article_content':'Text', 'summary':'Summary'}, inplace=True)

In [17]:
input_df = input_df[['Text', 'Summary']]

In [18]:
input_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Text     99 non-null     object
 1   Summary  99 non-null     object
dtypes: object(2)
memory usage: 1.7+ KB


In [19]:
input_df.head(10)

,Text,Summary
0,Traditional finance houses now being seriously...,Tencent has been granted a licence from the Ch...
1,"The Indian government's policy think tank, Nit...",India is testing blockchain applications in ed...
2,Increases in minimum wage levels risk raising ...,The UK Institute for Fiscal Studies has warned...
3,"Depending on whom you ask, WeWork is either a ...",Co-working start-up WeWork may go public this ...
4,AMD is big winner from chip flaw fiasco as mor...,Intel has seen more than $11bn wiped off its m...
5,"MUMBAI: About 250 companies, including some of...",Demand for cyber insurance among Indian compan...
6,Public sector organisations have quadrupled th...,The UK's public sector has increased its spend...
7,A worker pulls carts full of customer orders a...,The average price for large industrial plots o...
8,"Office take up in quarter four, 2017 in the ce...",Office take-up in Birmingham during 2017 broke...
9,Elevate your enterprise data technology and st...,"Machine learning chatbots – such as Ada, which..."


### Split the data into Train, Validation and Test dataset

In [20]:
from sklearn.model_selection import train_test_split

X_train, X_val = train_test_split(input_df, test_size = 0.2)

X_train, X_test = train_test_split(X_train, test_size = 0.1)

In [21]:
print(f'Shape of X_train is: {X_train.shape}')
#print(f'Shape of y_train is: {y_train.shape}')

print(f'Shape of X_val is: {X_val.shape}')
#print(f'Shape of y_val is: {y_val.shape}')

print(f'Shape of X_test is: {X_test.shape}')
#print(f'Shape of y_test is: {y_test.shape}')

Shape of X_train is: (71, 2)
Shape of X_val is: (20, 2)
Shape of X_test is: (8, 2)


***Saving the dataset for training, validation and test***

In [22]:
input_df.to_csv('curated_dataset_for_summarization.csv', index=False)
X_train.to_csv('training_data.csv', index=False)
X_val.to_csv('validation_data.csv', index=False)
X_test.to_csv('test_data.csv', index=False)

### Loading the dataset from Huggingface Dataset

In [23]:
from datasets import load_dataset

data_files = {"train": "training_data.csv", "val":"validation_data.csv", "test": "test_data.csv"}
dataset = load_dataset("sudhanshusinghaiml/curated-dataset-for-summarization", data_files= data_files)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [24]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Text', 'Summary'],
        num_rows: 71
    })
    val: Dataset({
        features: ['Text', 'Summary'],
        num_rows: 20
    })
    test: Dataset({
        features: ['Text', 'Summary'],
        num_rows: 8
    })
})

***Displaying data from loaded dataset***

In [25]:
dataset['train'][29]

{'Text': '"But, on the other hand, we would be foolish to rule anything out. We know that Asia-Pacific will be a very important market and we know a lot of the global growth in the future will come from there."',
 'Summary': 'According to the UK\'s International Trade Secretary, Liam Fox, the UK could feasibly join the Trans-Pacific Partnership (TPP), saying "it would be foolish to rule anything out". The organisation is made up of Australia, Mexico, New Zealand, Canada, Chile, Japan, Singapore, Brunei, Peru, Vietnam and Malaysia - with Donald Trump pulling the US out last year - and\xa0is currently in renegotiation under the new name of the Comprehensive and Progressive Agreement for Trans-Pacific Partnership. Its aims are to\xa0lower both non-tariff and tariff barriers to trade and to provide a forum to settle international disputes.'}

In [26]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [27]:
show_random_elements(dataset["train"])

In [28]:
from evaluate import load

metric = load("rouge")

In [29]:
metric

EvaluationModule(name: "rouge", module_type: "metric", features: [{'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id=None)}, {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}], usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLsum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/

### Tokenize the training, Validation and test dataset from AutoTokenizer

***Creating Model Checkpoint for better readability***

In [30]:
model_checkpoint = 'google/flan-t5-base'
max_input_length = 1024
max_target_length = 128
prefix = "Summarize: "

In [33]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

In [34]:
max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["Text"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=examples["Summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [35]:
preprocess_function(dataset['train'][:2])

{'input_ids': [[12198, 1635, 1737, 10, 18419, 1899, 349, 31260, 8666, 41, 4770, 11151, 61, 810, 3, 15503, 24, 8, 538, 13, 15625, 19, 8, 3, 14575, 538, 16, 8, 837, 12, 2000, 9310, 5, 4190, 7481, 5558, 2311, 3, 9, 418, 13, 2806, 579, 11, 8, 579, 1917, 7641, 16, 334, 538, 5, 6719, 30, 9265, 11151, 22, 7, 1251, 810, 13, 8, 583, 13, 19432, 5558, 640, 8, 837, 6, 34, 19, 1083, 3, 14575, 12, 2000, 9310, 16, 15625, 1636, 6373, 1358, 44, 5835, 4225, 3151, 7, 399, 3, 11876, 10762, 8, 1348, 583, 13, 5558, 80, 9310, 44, 5583, 6, 24622, 5, 100, 19, 4019, 8139, 145, 8, 750, 594, 13, 9310, 6, 84, 19, 1083, 3415, 44, 300, 1970, 8630, 399, 7485, 6, 38, 13, 2785, 97, 5, 2840, 1307, 16, 8, 837, 19, 34, 2877, 12, 2000, 58, 86, 70, 810, 6, 9265, 11151, 92, 5861, 8, 583, 13, 9310, 5558, 3, 390, 30, 8, 3, 11876, 545, 5962, 13, 8, 386, 167, 1012, 5558, 3, 3380, 7, 6, 3, 17332, 6, 8, 9141, 12858, 49, 180, 1298, 6, 8, 9141, 12858, 49, 180, 940, 6, 11, 8, 71, 2165, 106, 8580, 38, 168, 38, 8, 1348, 477, 284, 3, 33

In [36]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/71 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

### Train or Fine Tune with Pytorch Trainer on new dataset

We can download the pretrained model and fine-tune it. Since our task is of the sequence-to-sequence kind, we use the AutoModelForSeq2SeqLM class. Like with the tokenizer, the from_pretrained method will download and cache the model for us

In [37]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [43]:
!nvidia-smi

Thu Apr 18 17:59:45 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P0              27W /  70W |  14965MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [54]:
from accelerate import Accelerator

accelerator = Accelerator()

batch_size = 4
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    "sudhanshusinghaiml/facebook-bart-base-finetuned",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    gradient_accumulation_steps = 2
    # push_to_hub=True
)

We need a special kind of data collator, which will pad the inputs to the maximum length in the batch for all datasets and texts as well as summary

In [55]:
dataCollator = DataCollatorForSeq2Seq(tokenizer, model=model)

The last thing to define for our Seq2SeqTrainer is how to compute the metrics from the predictions. We need to define a function for this, which will just use the metric we loaded earlier, and we have to do a bit of pre-processing to decode the predictions into texts:

In [56]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

Then we just need to pass all of this along with our datasets to the Seq2SeqTrainer:

In [57]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["val"],
    data_collator=dataCollator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [58]:
import accelerate

accelerate.notebook_launcher(trainer.train)
# trainer.train()

Launching training on one GPU.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,nan,17.185700,5.473600,13.104400,15.626900,17.750000


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [59]:
# Save the model
trainer.save_model('google-flan-t5-base-finetuned')

Test the Preiction of the model

In [62]:
def summarize_from_t5_tuned_model(article):
    # Load sudhanshusinghaiml/google-flan-t5-base-fintuned model and tokenizer
    model_name = 'sudhanshusinghaiml/google-flan-t5-base-fintuned'
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

    # Tokenize and encode the article
    inputs = tokenizer(article, return_tensors='pt', max_length=1024, truncation=True)

    # Generate summary
    summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=128, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return summary

# Example usage
article = """
Former U.S. President Donald Trump charged in classified documents probe. There was no immediate confirmation from the Justice Department regarding Mr. Trump’s assertion, although some U.S. media outlet cited sources saying that the former U.S. President has bee indicted.
"""

summary = summarize_from_t5_tuned_model(article)
print("Summary:")
print(summary)

Summary:
Donald Trump has been charged in a classified documents probe.


In [63]:
article = """Packing all the contextual information from a document into a short summary is much harder with long text. If our summary has to be say 5 sentences max it is much harder to
decide what information is valuable enough to be added with 500 words vs 50,000 words. Chunking algorithms are often required, but they do grow the data variance coverage a model must have
to be accurate. Chunking algorithms control how much of the larger document we pass into a summarizer based on the max tokens the model allows and parameters we’ve set. The new dynamic
nature of the input data means our data variance is much larger than what is seen with smaller text. Longer documents often have much more internal data variance and swings in the information.
Use causes such as blog posts, interviews, transcripts and more have multiple swings in the dialog that make it harder to understand what contextual information is valuable for the summary.
Models have to learn a much deeper relationship between specific keywords, topics, and phrases as the text grows. There are two main types of summarization that are used as the baseline for
any enhanced versions of summarization - Extractive and abstractive. They focus on how the key information found in the input text is reconstructed in the generated summary in their own ways.
Both of these methods have their own unique challenges that pop up when looking at using longer text."""

summary = summarize_from_t5_tuned_model(article)
print("Summary:")
print(summary)

Summary:
Extractive and abstractive are the baselines for any enhanced versions of summarization.


In [60]:
def summarize_from_base_model(article):
    # Load BART model and tokenizer
    # model_name = 'sudhanshusinghaiml/facebook-bart-base-fintuned'
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

    # Tokenize and encode the article
    inputs = tokenizer(article, return_tensors='pt', max_length=1024, truncation=True)

    # Generate summary
    summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=128, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return summary

# Example usage
article = """
Former U.S. President Donald Trump charged in classified documents probe. There was no immediate confirmation from the Justice Department regarding Mr. Trump’s assertion, although some U.S. media outlet cited sources saying that the former U.S. President has bee indicted.
"""

summary = summarize_from_base_model(article)
print("Summary:")
print(summary)

Summary:
Donald Trump has been charged in a classified documents probe.


In [61]:
article = """Packing all the contextual information from a document into a short summary is much harder with long text. If our summary has to be say 5 sentences max it is much harder to
decide what information is valuable enough to be added with 500 words vs 50,000 words. Chunking algorithms are often required, but they do grow the data variance coverage a model must have
to be accurate. Chunking algorithms control how much of the larger document we pass into a summarizer based on the max tokens the model allows and parameters we’ve set. The new dynamic
nature of the input data means our data variance is much larger than what is seen with smaller text. Longer documents often have much more internal data variance and swings in the information.
Use causes such as blog posts, interviews, transcripts and more have multiple swings in the dialog that make it harder to understand what contextual information is valuable for the summary.
Models have to learn a much deeper relationship between specific keywords, topics, and phrases as the text grows. There are two main types of summarization that are used as the baseline for
any enhanced versions of summarization - Extractive and abstractive. They focus on how the key information found in the input text is reconstructed in the generated summary in their own ways.
Both of these methods have their own unique challenges that pop up when looking at using longer text."""

summary = summarize_from_base_model(article)
print("Summary:")
print(summary)

Summary:
Extractive and abstractive are the baselines for any enhanced versions of summarization.


***We can clearly see that the performance of google/flan-t5-base model is as good as that of google/flan-t5-base model fintuned with custom dataset***